# DataProcessing Notebook - AAAG

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
import statsmodels.api as sm
pd.set_option('display.max_columns', None)
df_train=pd.read_csv('Version_5_df.csv',index_col='Id')

In [2]:
df_train.sample(10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
820,120,RL,44.0,6371,Pave,No Alley,IR1,Lvl,Inside,Gtl,NridgHt,Norm,Norm,TwnhsE,1Story,7,5,2009,2010,Gable,CompShg,VinylSd,VinylSd,Stone,128.0,4,3,PConc,4,3,2,GLQ,1358,GasA,5,Y,SBrkr,1358,0,1358,1,0,2,0,2,4,6,Typ,1,4,Attchd,2010.000000,RFn,2,484,3,3,Y,1,1,0,0,0,0,No Fence,No Shed,6,2010,New,Partial,12.319406
1273,20,RL,71.0,13526,Pave,No Alley,IR1,Lvl,CulDSac,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,6,1965,1965,Hip,CompShg,HdBoard,Plywood,BrkFace,114.0,3,3,CBlock,3,3,1,BLQ,935,GasA,3,Y,SBrkr,935,0,935,1,0,1,0,3,3,5,Typ,0,0,Attchd,1965.000000,Unf,1,288,3,3,Y,1,0,0,0,0,0,MnPrv,No Shed,11,2006,WD,Normal,11.827744
1022,20,RL,64.0,7406,Pave,No Alley,Reg,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,Stone,84.0,4,3,PConc,4,3,3,GLQ,1199,GasA,5,Y,SBrkr,1220,0,1220,1,0,2,0,2,4,6,Typ,0,0,Attchd,2006.000000,RFn,2,632,3,3,Y,1,1,0,0,0,0,No Fence,No Shed,7,2006,New,Partial,12.175619
1408,20,RL,73.0,8780,Pave,No Alley,IR1,Lvl,Corner,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,5,1985,1985,Gable,CompShg,HdBoard,Plywood,None,0.0,3,3,CBlock,3,3,1,ALQ,833,GasA,5,Y,SBrkr,833,0,833,1,0,1,0,3,3,5,Typ,0,0,No Garage,1978.506164,No Garage,0,0,0,0,Y,0,0,0,0,0,0,MnPrv,No Shed,3,2009,WD,Normal,11.626263
713,120,RL,40.0,4671,Pave,No Alley,IR1,HLS,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1988,1989,Gable,CompShg,HdBoard,HdBoard,None,0.0,4,3,PConc,4,3,2,GLQ,1228,GasA,4,Y,SBrkr,1228,0,1228,1,0,2,0,2,4,5,Typ,1,4,Attchd,1988.000000,Fin,2,472,3,3,Y,1,1,0,0,0,0,No Fence,No Shed,10,2008,WD,Normal,12.149508
1450,180,RM,21.0,1533,Pave,No Alley,Reg,Lvl,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,SFoyer,5,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,3,3,CBlock,4,3,3,GLQ,630,GasA,5,Y,SBrkr,630,0,630,1,0,1,0,1,5,3,Typ,0,0,No Garage,1978.506164,No Garage,0,0,0,0,Y,0,0,0,0,0,0,No Fence,No Shed,8,2006,WD,Abnorml,11.429555
491,160,RM,21.0,2665,Pave,No Alley,Reg,Lvl,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,5,6,1976,1976,Gable,CompShg,CemntBd,CmentBd,None,0.0,3,3,PConc,4,3,2,Unf,264,GasA,3,Y,SBrkr,616,688,1304,0,0,1,1,3,3,4,Typ,1,4,BuiltIn,1976.000000,Fin,1,336,3,3,Y,1,1,0,0,0,0,No Fence,No Shed,6,2008,WD,Normal,11.652696
153,60,RL,80.0,14803,Pave,No Alley,IR1,Lvl,CulDSac,Gtl,NWAmes,Norm,Norm,1Fam,2Story,6,5,1971,1971,Gable,CompShg,HdBoard,HdBoard,BrkFace,252.0,3,3,CBlock,3,3,1,Rec,825,GasA,4,Y,SBrkr,1097,896,1993,0,0,2,1,4,3,8,Typ,1,4,Attchd,1971.000000,RFn,2,495,3,3,Y,0,1,0,0,0,0,GdWo,No Shed,6,2006,WD,Normal,12.154785
1212,50,RL,152.0,12134,Pave,No Alley,IR1,Bnk,Inside,Mod,Gilbert,Norm,Norm,1Fam,1.5Fin,8,7,1988,2005,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,4,3,Wood,4,3,3,GLQ,559,GasA,4,Y,SBrkr,1080,672,1752,0,0,2,0,4,3,8,Typ,0,0,Basment,1988.000000,RFn,2,492,3,3,Y,1,1,0,0,0,0,No Fence,No Shed,6,2010,WD,Normal,12.007628


In [3]:
###########################################################################
#### Just for reference #####
###########################################################################
# Original category type casting: 
nominal_var=['MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope',\
             'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',\
             'Exterior1st','Exterior2nd','MasVnrType','Foundation',\
             'BsmtFinType1','BsmtFinType2','Heating','CentralAir',\
             'Electrical','Functional','GarageType','GarageFinish',\
             'PavedDrive','Fence','MiscFeature','SaleType','SaleCondition','MSSubClass']
ordinal_var=['OverallQual','OverallCond','BsmtFullBath','BsmtHalfBath','FullBath',\
             'HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars',\
             'MoSold','YrSold','HeatingQC','KitchenQual','ExterQual','ExterCond','BsmtQual',\
             'BsmtCond','BsmtExposure','FireplaceQu','GarageQual','GarageCond','PoolQC']

cont_var=['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF',\
          '1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','GarageYrBlt','GarageArea','WoodDeckSF','OpenPorchSF',\
          'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal']

var_dict={'nominal_var':nominal_var,'ordinal_var':ordinal_var,'cont_var':cont_var}
###########################################################################
#### Just for reference #####
###########################################################################


In [4]:
def feature_standardize(data,scaleType='standardize'):
    '''
    - Accepts a dataframe column
    '''
    if scaleType not in ['standardize', 'normalize']: 
        raise ValueError('%s is not a valid choice' %(scaleType))
    mean_value=np.mean(data)
    standard_dev=np.std(data)
    min_value=np.min(data)
    max_value=np.min(data)
    if scaleType == 'standardize':
        return((data-mean_value)/standard_dev) 
    elif scaleType == 'normalize':
        return((data-min_value)/(max_value-min_value))

In [5]:
def dummify_column(dataframe,column_name):
    '''
    ### NOTE!!! ### vector operation not working yet!!!! ###
    - dataframe takes the entire dataframe you are working on
    - column_name takes a list of strings, where the strings are the column names
    '''
    for feature in column_name:
        dummified_feature = pd.get_dummies(dataframe.loc[:,feature], prefix=feature, prefix_sep='__',drop_first=True)
        dummified_df = pd.concat([dataframe.drop(feature,axis=1),dummified_feature],axis=1,sort='False')
    return dummified_df

In [6]:
def delete_outliers(dataframe,column_name,threshold=5):
    '''
    - Accepts a full dataframe
    - Specify column_name to indicate which columns to check for outliers
    - Column name accepts list of strings, where the strings are the column names
    - threshold is the number of deviations that will be used for removing outliers
    '''
    for feature in column_name:
        standard_dev=np.std(dataframe.loc[:,feature])
        dataframe=dataframe.drop(dataframe[dataframe.loc[:,feature]>standard_dev*threshold].index,axis=0)
    return dataframe


In [7]:
# Check to see if nominal_var is contained in df_train
nominal_var_processed=['MSZoning','Street','Alley','LotShape','LandContour','LotConfig','LandSlope',\
             'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',\
             'Exterior1st','Exterior2nd','MasVnrType','Foundation',\
             'BsmtFinType1','Heating','CentralAir',\
             'Electrical','Functional','GarageType','GarageFinish',\
             'PavedDrive','Fence','MiscFeature','SaleType','SaleCondition','MSSubClass',\
             'WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea']
i=0
for feature in nominal_var_processed:
    if not df_train.loc[:,feature].empty:
        i+=1
np.array(nominal_var_processed).size==i

True

In [8]:
# dummify the nominal categorical columns
df_train_copy=df_train.copy()
for feature in nominal_var_processed:
    df_train_copy=dummify_column(df_train_copy,[feature])
dummified_df=df_train_copy
print(dummified_df.shape)
print(dummified_df.isnull().sum().sum())
dummified_df.head(10)

(1460, 224)
0


,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageYrBlt,GarageCars,GarageArea,GarageQual,GarageCond,MoSold,YrSold,SalePrice,MSZoning__FV,MSZoning__RH,MSZoning__RL,MSZoning__RM,Street__Pave,Alley__No Alley,Alley__Pave,LotShape__IR2,LotShape__IR3,LotShape__Reg,LandContour__HLS,LandContour__Low,LandContour__Lvl,LotConfig__CulDSac,LotConfig__FR2,LotConfig__FR3,LotConfig__Inside,LandSlope__Mod,LandSlope__Sev,Neighborhood__Blueste,Neighborhood__BrDale,Neighborhood__BrkSide,Neighborhood__ClearCr,Neighborhood__CollgCr,Neighborhood__Crawfor,Neighborhood__Edwards,Neighborhood__Gilbert,Neighborhood__IDOTRR,Neighborhood__MeadowV,Neighborhood__Mitchel,Neighborhood__NAmes,Neighborhood__NPkVill,Neighborhood__NWAmes,Neighborhood__NoRidge,Neighborhood__NridgHt,Neighborhood__OldTown,Neighborhood__SWISU,Neighborhood__Sawyer,Neighborhood__SawyerW,Neighborhood__Somerst,Neighborhood__StoneBr,Neighborhood__Timber,Neighborhood__Veenker,Condition1__Feedr,Condition1__Norm,Condition1__PosA,Condition1__PosN,Condition1__RRAe,Condition1__RRAn,Condition1__RRNe,Condition1__RRNn,Condition2__Feedr,Condition2__Norm,Condition2__PosA,Condition2__PosN,Condition2__RRAe,Condition2__RRAn,Condition2__RRNn,BldgType__2fmCon,BldgType__Duplex,BldgType__Twnhs,BldgType__TwnhsE,HouseStyle__1.5Unf,HouseStyle__1Story,HouseStyle__2.5Fin,HouseStyle__2.5Unf,HouseStyle__2Story,HouseStyle__SFoyer,HouseStyle__SLvl,RoofStyle__Gable,RoofStyle__Gambrel,RoofStyle__Hip,RoofStyle__Mansard,RoofStyle__Shed,RoofMatl__CompShg,RoofMatl__Membran,RoofMatl__Metal,RoofMatl__Roll,RoofMatl__Tar&Grv,RoofMatl__WdShake,RoofMatl__WdShngl,Exterior1st__AsphShn,Exterior1st__BrkComm,Exterior1st__BrkFace,Exterior1st__CBlock,Exterior1st__CemntBd,Exterior1st__HdBoard,Exterior1st__ImStucc,Exterior1st__MetalSd,Exterior1st__Plywood,Exterior1st__Stone,Exterior1st__Stucco,Exterior1st__VinylSd,Exterior1st__Wd Sdng,Exterior1st__WdShing,Exterior2nd__AsphShn,Exterior2nd__Brk Cmn,Exterior2nd__BrkFace,Exterior2nd__CBlock,Exterior2nd__CmentBd,Exterior2nd__HdBoard,Exterior2nd__ImStucc,Exterior2nd__MetalSd,Exterior2nd__Other,Exterior2nd__Plywood,Exterior2nd__Stone,Exterior2nd__Stucco,Exterior2nd__VinylSd,Exterior2nd__Wd Sdng,Exterior2nd__Wd Shng,MasVnrType__BrkFace,MasVnrType__No Masonry Veneer,MasVnrType__None,MasVnrType__Stone,Foundation__CBlock,Foundation__PConc,Foundation__Slab,Foundation__Stone,Foundation__Wood,BsmtFinType1__BLQ,BsmtFinType1__GLQ,BsmtFinType1__LwQ,BsmtFinType1__No Basement,BsmtFinType1__Rec,BsmtFinType1__Unf,Heating__GasA,Heating__GasW,Heating__Grav,Heating__OthW,Heating__Wall,CentralAir__Y,Electrical__FuseF,Electrical__FuseP,Electrical__Mix,Electrical__SBrkr,Functional__Maj2,Functional__Min1,Functional__Min2,Functional__Mod,Functional__Sev,Functional__Typ,GarageType__Attchd,GarageType__Basment,GarageType__BuiltIn,GarageType__CarPort,GarageType__Detchd,GarageType__No Garage,GarageFinish__No Garage,GarageFinish__RFn,GarageFinish__Unf,PavedDrive__P,PavedDrive__Y,Fence__GdWo,Fence__MnPrv,Fence__MnWw,Fence__No Fence,MiscFeature__Shed,SaleType__CWD,SaleType__Con,SaleType__ConLD,SaleType__ConLI,SaleType__ConLw,SaleType__New,SaleType__Oth,SaleType__WD,SaleCondition__AdjLand,SaleCondition__Alloca,SaleCondition__Family,SaleCondition__Normal,SaleCondition__Partial,MSSubClass__30,MSSubClass__40,MSSubClass__45,MSSubClass__50,MSSubClass__60,MSSubClass__70,MSSubClass__75,MSSubClass__80,MSSubClass__85,MSSubClass__90,MSSubClass__120,MSSubClass__160,MSSubClass__180,MSSubClass__190,WoodDeckSF__1,OpenPorchSF__1,EnclosedPorch__1,3SsnPorch__1,ScreenPorch__1,PoolArea__1
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [9]:
# For all continuous features, remove outliers where outlier >= 5 std
dummified_df_copy=dummified_df.copy()
for feature in ['LotFrontage','LotArea','MasVnrArea','TotalBsmtSF','1stFlrSF','2ndFlrSF','GrLivArea','GarageArea']:
    dummified_df_copy=delete_outliers(dummified_df_copy,[feature],threshold=5)
baseline_df=dummified_df_copy 
baseline_df.shape

(1215, 224)

In [10]:
# Check if ordinal variables are still there:
ordinal_var_processed=['OverallQual','OverallCond','BsmtFullBath','BsmtHalfBath','FullBath',\
             'HalfBath','BedroomAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars',\
             'MoSold','YrSold','HeatingQC','KitchenQual','ExterQual','ExterCond','BsmtQual',\
             'BsmtCond','BsmtExposure','FireplaceQu','GarageQual','GarageCond']

i=0
for feature in ordinal_var_processed:
    if not baseline_df.loc[:,feature].empty:
        i+=1
np.array(ordinal_var_processed).size==i

True

In [11]:
# Check if continuous variables are still there:
cont_var_processed=['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea','TotalBsmtSF',\
          '1stFlrSF','2ndFlrSF','GrLivArea','GarageYrBlt','GarageArea']
i=0
for feature in cont_var_processed:
    if not baseline_df.loc[:,feature].empty:
        i+=1
np.array(cont_var_processed).size==i

True

In [12]:
standardize_cont_var=ordinal_var_processed+cont_var_processed
standardized_baseline_df=baseline_df.copy()
standardized_baseline_df[standardize_cont_var]=baseline_df[standardize_cont_var].apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)
normalized_baseline_df=baseline_df.copy()
normalized_baseline_df[standardize_cont_var]=baseline_df[standardize_cont_var].apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)
print(normalized_baseline_df.shape)
print(standardized_baseline_df.shape)

(1215, 224)
(1215, 224)


In [13]:
baseline_df.to_csv('baseline_df.csv')
standardized_baseline_df.to_csv('standardized_baseline_df.csv')
normalized_baseline_df.to_csv('normalized_baseline_df.csv')